# ELECTRA: Pre-training Text Encoders as Discriminators Rather Than Generators

* URL: https://arxiv.org/abs/2003.10555
* authers: Kevin Clark, Minh-Thang Luong, Quoc V. Le, Christopher D. Manning
* Submitted on 23 Mar 2020

### 1. どんなもの？
  新しい言語表現モデルであるELECTRA,  which stands for **E**fficiently **L**earning an **E**ncoder that **C**lassifies **T**oken **R**eplacements **A**ccurately の紹介.

### 2. 先行研究と比べるとどこが凄い？
* 先行研究であるBERTやRoBERTaのようなMasked Language Modeling pre-trained methodとは異なってreplacing token detectionを導入することによって、訓練効率を上昇させた。

### 3. 技術や手法のキモはどこ？
* Replacing token detection  
モデルをgeneratorとdiscriminatorの2つに分けた。inputに一部[MASK]をして、そのtokenをgeneratorによって最もそれらしい(尤度が最大の)tokenに置き換え(generatorが[MASK]前と同じtokenを置き換えたらそのままのtokenを出すようにした)、discriminatorに入力。そこでどのtokenが置き換えられているのかを予測するようにして学習させた。generator部分ではback-propagateは行わない。pre-trainedが終わったらgenerator部分は使わず、discriminator部分のみでdownstream taskに対してfine-tuningを行う。損失関数は以下の通り。詳しいことは論文参照。

$$
\begin{cases}
 L_{MLM}(x, \theta_{G}) = E\left(\sum_{i \in m}-\log_{p_{G}}(x_{i}|x^{masked})\right)\\
 L_{Disc}(x, \theta_{D}) = E\left(\sum_{t=1}^{n}-1(x_{t}^{corrupt}=x_{t})\log(D(x^{corrupt}, t)) -1(x_{t}^{corrupt} \ne x_{t})\log(1-D(1-D(x^{corrupt}, t)))\right)\\
 \min_{\theta_{G}, \theta_{D}}\sum_{x\in X} L_{MLM}(x, \theta_{G}) + \lambda L_{Disc}{x, \theta_{D}}\\
 D = softmax\\
 p = simgoid
\end{cases}
$$

* Weight Sharing  
訓練効率を改善させるためにgeneratorとdiscriminetorでweightを共有することにした。実験ではdiscriminetorよりもlayer数が少なく、サイズが1/2~1/4のgeneratorを用いると精度が上昇したので、embeddings(token and positional embeddings)のみを共有することにした。

* 小さいモデルでも大きいモデルでも性能が高かった。


### 4. どうやって有効だと検証した？
GLUEとSQuADでモデルのサイズや訓練回数、評価指標に対する評価を10のseedのmedianを取って比較。

### 5. 議論はある？
* ELECTRAを多言語にするとどうなる？
* ELECTRAのgeneratorで、MLMではなくauto-regressiveにしたらどうなるのか？
* 大量のデータを使うとどうなる？


### 6. 次に読むべき論文は？
* [DeBERTa: Decoding-enhanced BERT with Disentangled Attention](https://arxiv.org/abs/2006.03654)
* MASS: auto-regressive
* UniLM: auto-regressive

### 7. 実装
https://github.com/google-research/electra